In [79]:
import numpy as np
from abc import ABC, abstractmethod
from itertools import zip_longest

# Fields
Field: we have a set, two operations ("addition" and "multiplication") and this is a commutative group w.r.t. both operations, plus multiplication distributes over addition.

### $F_p$ for $p \in \mathbb{P}$
Basic example of a finite field is $\mathbb{Z} / p \mathbb{Z}$, i.e. integers modulo a prime number $p$. 

In [46]:
# full definition would include 0 and 1 (neutral elements), but in our considerations they will be literally 0 and 1
class FieldOps(ABC):
    @abstractmethod
    def add(self, a, b):
        pass

    @abstractmethod
    def add_inv(self, a):
        pass

    def subtract(self, a, b):
        return self.add(a, self.add_inv(b))
    
    @abstractmethod
    def mult(self, a, b):
        pass

    @abstractmethod
    def mult_inv(self, a):
        pass

Bezout's identity says that $gcd(a, b) = a s + b t$ for some $s, t \in \mathbb{Z}$, so if they are coprime (e.g. $a \in \mathbb{P}$ and $b \in \{1, ..., a - 1 \}$), then $b t = 1 \ (mod \ p)$, so $n$ is the inverse. Extended Euclidean algorithm is used to find Bezout's coefficients and hence the inverse. 

For example:
 - $240 - 5 * 46 = 10$
 - $46 - 4 * 10 = 6$
 - $10 - 1 * 6 = 4$
 - $6 - 1 * 4 = 2$
 - $4 - 2 * 2 = 0$ (hence $gcd(240, 46) = 2$ because that's the last non-zero remainder)
 
We can start from the GCD and "go back" using division equations to present it as in Bezout's identity:

$$
\begin{align}
2 & = 6 - 1 * 4 = 6 - (10 - 6) = 2 * 6 - 10 \\& = 2 * (46 - 4 * 10) - 10 = 2 * 46 - 9 * 10 \\& = 2 * 46 - 9 * (240 - 5 * 46) = 47 * 46 - 9 * 240
\end{align}
$$

Luckily, [the real extended Euclidean algorithm](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm) is smarter than this and doesn't include this going back.

In [47]:
def ext_euclidean(a, b):
    # a > b
    # r_{i - 1}, r_i starting from i = 1
    r_i_m_1, r_i = a, b

    # a * s_i + b * t_i = r_i for all i, definition of s_i and t_i so smart! 
    s_i_m_1, t_i_m_1 = 1, 0
    s_i, t_i = 0, 1  

    while r_i != 0:
        # r_{i + 1} = r_{i - 1} - q_i * r_i
        r_i_p_1 = r_i_m_1 % r_i
        q_i = r_i_m_1 // r_i
        
        s_i_p_1 = s_i_m_1 - q_i * s_i
        t_i_p_1 = t_i_m_1 - q_i * t_i

        r_i_m_1, r_i = r_i, r_i_p_1
        s_i_m_1, s_i = s_i, s_i_p_1
        t_i_m_1, t_i = t_i, t_i_p_1

    return r_i_m_1, s_i_m_1, t_i_m_1

In [48]:
gcd, s, t = ext_euclidean(240, 46)
assert 240 * s + 46 * t == gcd
print((gcd, s, t))

(2, -9, 47)


In [49]:
def ff_for_prime(p):
    class Ops(FieldOps):
        def add(self, a, b):
            return (a + b) % p

        def add_inv(self, a):
            return (-a) % p

        def mult(self, a, b):
            return (a * b) % p

        def mult_inv(self, a):
            _, _, inv = ext_euclidean(p, a)
            return inv

    return Ops()

In [50]:
f_31 = ff_for_prime(31)

for i in range(1, 31):
    assert f_31.mult(i, f_31.mult_inv(i)) == 1

### Extension to $F_{p^m}$ for $p \in \mathbb{P}$
For ease of use, we will still represent field elements by numbers in $\{0, ..., p^m - 1\}$, but operations on them will no longer be ordinary $(mod \ p)$ operations. Each number can be viewed as its $m$-length representation in base $p$, i.e. $x = a_{m - 1} p^{m - 1} + ... + a_0 p^0$. This vector $(a_{m-1}, ..., a_0)$ can in turn be viewed as  coefficients of a polynomial of degree $\lt m$ over the base field $F_p$. So really the elements of the field will be these polynomials, it's just nicer to represent them as numbers and define new addition and multiplication tables for them.

Then we take an irreducible polynomial $p(x)$ over $F_p$ (i.e. it cannot be factorized using coefficients in this field) of degree $m$ and define all operations as polynomial operations $(mod \ p(x))$ - and that's it! Existence of multiplicative inverse is satisfied by using extended Euclidean algorithm again. Importantly, such polynomials exist for all pairs of prime $p$ and $m$.

In [185]:
# a, b: lists of coefficients in given finite field [c_0, ..., c_k], c_k != 0
def add_poly(a, b, field):
    zipped = zip_longest(a, b, fillvalue=0)  # again, 0 is zero
    return [field.add(a, b) for (a, b) in zipped]


def subtract_poly(a, b, field):
    zipped = zip_longest(a, b, fillvalue=0)  # again, 0 is zero
    return [field.subtract(a, b) for (a, b) in zipped]


def mult_poly(a, b, field):
    deg_a, deg_b = len(a) - 1, len(b) - 1
    result = [0] * (deg_a + deg_b + 1)  # using the fact that number zero actually represents zero in all our finite fields

    for i in range(deg_a + 1):
        for j in range(deg_b + 1):
            coef = field.mult(a[i], b[j])
            position = i + j
            result[position] = field.add(result[position], coef)

    return result


def div_poly(a, b, field):
    # careful - highest degree coefficient must be non-zero
    deg_a, deg_b = len(a) - 1, len(b) - 1    
    quotient, remainder = [], a.copy()

    # required to divide a's coefficients by b's highest coefficient
    # division always works, because it's a field
    b_coef_inv = field.mult_inv(b[-1])

    for i in range(deg_a - deg_b + 1):
        multiplier = field.mult(remainder[-(i + 1)], b_coef_inv)
        quotient.append(multiplier)  # needs to be reversed at the end

        for j in range(deg_b + 1):
            coef = field.mult(multiplier, b[deg_b - j])
            remainder[-(i + 1) - j] = field.subtract(remainder[-(i + 1) - j], coef)

    # remove trailing zeros from remainder (crucial for extended Euclidean algorithm to work
    # since we pass it to div_poly again)
    non_zero_indexes = [i for i, r_i in enumerate(remainder) if r_i != 0]
    return quotient[::-1] if quotient else [0], remainder[:non_zero_indexes[-1] + 1] if non_zero_indexes else [0]

In [188]:
f_2 = ff_for_prime(2)

# (x + 1)^2 = x^2 + 1  // Freshman's dream <3
assert mult_poly([1, 1], [1, 1], f_2) == [1, 0, 1]

# x^2 + x + 1 = x * (x + 1) + 1
assert div_poly([1, 1, 1], [1, 1], f_2) == ([0, 1], [1])

In [189]:
def ext_euclidean_poly(a, b, field):
    deg_a, deg_b = len(a) - 1, len(b) - 1
    assert deg_a > deg_b
    # r_{i - 1}, r_i starting from i = 1
    r_i_m_1, r_i = a, b

    # a * s_i + b * t_i = r_i for all i
    # again, using the fact that 0 and 1 represent neutral elements of our finite fields!
    s_i_m_1, t_i_m_1 = [1], [0]
    s_i, t_i = [0], [1]

    # iterate until we get a pair where one polynomials divides the other
    while any(element != 0 for element in r_i):
        import time
        time.sleep(1)
        # r_{i + 1} = r_{i - 1} - q_i * r_i
        q_i, r_i_p_1 = div_poly(r_i_m_1, r_i, field)
        # print(r_i_m_1, r_i, q_i, r_i_p_1)

        s_i_p_1 = subtract_poly(s_i_m_1, mult_poly(q_i, s_i, field), field)
        t_i_p_1 = subtract_poly(t_i_m_1, mult_poly(q_i, t_i, field), field)
        
        r_i_m_1, r_i = r_i, r_i_p_1
        s_i_m_1, s_i = s_i, s_i_p_1
        t_i_m_1, t_i = t_i, t_i_p_1

    return r_i_m_1, s_i_m_1, t_i_m_1

In [191]:
# following https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm
assert ext_euclidean_poly([1, 1, 0, 1, 1, 0, 0, 0, 1], [1, 1, 0, 0, 1, 0, 1], f_2) == (
    [1], [1, 0, 1, 1, 1, 1, 0, 0], [0, 1, 0, 1, 0, 0, 1, 1]
)

Wow, that's SLOW. But it works!

In [198]:
# p: order of the base field
# field: operations on elements of the base field (represented as numbers 0, ..., p - 1)
# irr_p: np.array of irreducible polynomial coefficients, [p_0, ..., p_m]
def ff_for_irreducible_polynomial(p, field, irr_p):
    # number to polynomial
    def n2p(n):
        poly = []

        while n != 0:
            poly.append(n % p)
            n //= p

        assert len(poly) < len(irr_p), f'{n} is too large for a correct representation of this field\'s element!'
        return poly


    # polynomial to number
    def p2n(poly):
        return sum(p_i * p ** i for i, p_i in enumerate(poly))
        
    
    class Ops(FieldOps):
        def add(self, a, b):
            a_p, b_p = n2p(a), n2p(b)
            return p2n(add_poly(a_p, b_p, field))

        def add_inv(self, a):
            a_p = n2p(a)
            return p2n([field.add_inv(a_i) for a_i in a_p])

        def mult(self, a, b):
            a_p, b_p = n2p(a), n2p(b)
            ab_p = mult_poly(a_p, b_p, field)
            q, rem = div_poly(ab_p, irr_p, field)
            return p2n(rem)

        def mult_inv(self, a):
            _, _, inv = ext_euclidean_poly(irr_p, n2p(a), field)
            return p2n(inv)

    return Ops()

In [199]:
# x^4 + x + 1 is irreducible over F_2 - appendix C

# let's add 1101 and 0111, i.e. 11 and 14 - should produce 1010, i.e. 5
f_16 = ff_for_irreducible_polynomial(2, f_2, [1, 1, 0, 0, 1])
assert f_16.add(11, 14) == 5

# now let's multiply them - should produce 0001, i.e. 8 (AWESOME tool to verify: http://www.ee.unb.ca/cgi-bin/tervo/galois3.pl?p=4)
assert f_16.mult(11, 14) == 8

Let us also test the whole multiplication table, as correctness is quite important here.

In [200]:
def mult_table(q, field):
    table = [[0] * q for _ in range(q)]
    
    for a in range(q):
        for b in range(q):
            table[a][b] = field.mult(a, b)
    return table


# http://www.ee.unb.ca/cgi-bin/tervo/galois3.pl?p=4 strikes again!
# reference string can be copied directly from the Web page, starting with 'x'
def mult_table_from_ref(ref_str):
    ref_mult_table = [[0] * 16 for _ in range(16)]
    ref_mult_table_str_elements = ref_mult_table_str.split()
    for i in range(16):
        for j in range(16):
            # ignore 1st row and column
            if i != 0 and j != 0:
                ref_mult_table[i][j] = int(ref_mult_table_str_elements[16 * i + j])
    return ref_mult_table


ref_mult_table_str = """
x	1 	2 	3 	4 	5 	6 	7 	8 	9 	10 	11 	12 	13 	14 	15
1	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15
2	2	4	6	8	10	12	14	3	1	7	5	11	9	15	13
3	3	6	5	12	15	10	9	11	8	13	14	7	4	1	2
4	4	8	12	3	7	11	15	6	2	14	10	5	1	13	9
5	5	10	15	7	2	13	8	14	11	4	1	9	12	3	6
6	6	12	10	11	13	7	1	5	3	9	15	14	8	2	4
7	7	14	9	15	8	1	6	13	10	3	4	2	5	12	11
8	8	3	11	6	14	5	13	12	4	15	7	10	2	9	1
9	9	1	8	2	11	3	10	4	13	5	12	6	15	7	14
10	10	7	13	14	4	9	3	15	5	8	2	1	11	6	12
11	11	5	14	10	1	15	4	7	12	2	9	13	6	8	3
12	12	11	7	5	9	14	2	10	6	1	13	15	3	4	8
13	13	9	4	1	12	8	5	2	15	11	6	3	14	10	7
14	14	15	1	13	3	2	12	9	7	6	8	4	10	11	5
15	15	13	2	9	6	4	11	1	14	12	3	8	7	5	10"""



assert mult_table(16, f_16) == mult_table_from_ref(ref_mult_table_str)

Yeah! It's so nice that the reference website also represents polynomials with numbers the way we do - appendix C maps polynomials using powers of a primitive element.

As a final test let's see if multiplicative inverses work.

In [201]:
for i in range(1, 16):
    assert f_16.mult(i, f_16.mult_inv(i)) == 1

So slow - but works, so still prety cool.

This construction works if we replace $F_p$ by any finite field actually, so we can use the field of polynomials as base and then extend it. Therefore if we have a finite field $F_{q^m}$ and $n = m \ k$ for some $k$, then we can build $F_{q^n}$ from $k$-degree polynomials with coefficients in $F_{q^m}$. $F_{q^m}$ is then a *subfield* of $F_{q^n}$ if we consider operations on just 0-degree polynomials. Looking at the multiplication table of $F_{q^n}$ it's just gonna be a subset where multiplication works fine.

Another fun fact is that all finite fields have orders of form $q = p^m, \ p \in \mathbb{P}$ and all finite fields of given order are isomorphic. Hence they get a special name and notation - Galois fields $GF(q)$ (after their discoverer).

### Additional concepts
**Primitive root $\alpha$** - generator of multiplicative group of non-zero elements of $F_q$, i.e. $\forall_{x \in F_q, \ x \neq 0} \ \exists_{k \in \mathbb{N}}: \ x = \alpha^k$

**Minimal polynomial** $f(x)$ of element $\alpha \in F_{q^m}$ - monic (highest power coefficient = 1) polynomial with coefficients in $F_q$ of the lowest degree s.t. $f(\alpha) = 0$. The multiplication of $\alpha \in F_{q^m}$ by coefficients in $F_q$ works because $F_q$ is a subfield of $F_{q^m}$.